<a href="https://colab.research.google.com/github/mdarafay7/ALGORITHMS/blob/master/6363_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install surprise
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
 
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.model_selection import train_test_split
from google.colab import files
from surprise import Reader, Dataset, KNNBasic, CoClustering
from surprise.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
 
 
uploaded = files.upload()
movies = pd.read_csv("movies.csv")
ratings = pd.read_csv("ratings.csv")
links = pd.read_csv("links.csv")
tags = pd.read_csv("tags.csv")

     |████████████████████████████████| 11.8MB 278kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617597 sha256=2b61731b58a6c3fda74de05c19f9121d5a3b4733dcf24a5cdc1fd778b0106528
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


Saving links.csv to links.csv
Saving movies.csv to movies.csv
Saving ratings.csv to ratings.csv
Saving tags.csv to tags.csv


In [ ]:
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split

In [ ]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)
trainset = data.build_full_trainset()
testset = trainset.build_anti_testset()


In [ ]:
from collections import defaultdict

def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [ ]:


trainset = data.build_full_trainset()
algorithm = CoClustering()
algorithm.fit(trainset)
testset = trainset.build_anti_testset()
predictions = algorithm.test(testset)
accuracy.rmse(predictions, verbose=True) 



RMSE: 1.0045


1.004548359313665

In [ ]:
algorithm = KNNBasic()
algorithm.fit(trainset)
predictions = algorithm.test(testset)
accuracy.rmse(predictions, verbose=True) 

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8870


0.8870334282999758

In [ ]:
algorithm = SVD()
algorithm.fit(trainset)
testset = trainset.build_anti_testset()
predictions = algorithm.test(testset)
accuracy.rmse(predictions, verbose=True) 

RMSE: 0.4858


0.48582668114926225

In [ ]:
knn = KNNBasic(k = 100)

trainset = data.build_full_trainset()

knn.fit(trainset)

testset = trainset.build_anti_testset()
predictions = knn.test(testset)
top_n = get_top_n(predictions, n = 10)

accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8864


0.8863717951719997

In [ ]:
for uid, user_ratings in top_n.items():
    x=[]
    for (iid, _) in user_ratings:
      p = movies["title"].loc[movies["movieId"]==iid].to_string(index = False)
      x.append(p)

for movie in x:
  print(movie)


 The Jinx: The Life and Deaths of Robert Durst ...
 Galaxy of Terror (Quest) (1981)
 Alien Contamination (1980)
 I'm the One That I Want (2000)
 Lesson Faust (1994)
 Assignment, The (1997)
 Mephisto (1981)
 Black Mirror
 Dylan Moran: Monster (2004)
 Bill Hicks: Revelations (1993)
